In [ ]:
import re
import unicodedata
import pandas as pd
from pathlib import Path

path_source = "Lexicc - CaroCuervo/Diccionario_breve_de_Colombiaismos_slown_text.txt"

#  limpieza 
def nfc(s: str) -> str:
    return unicodedata.normalize("NFC", s or "")

def norm_spaces(s: str) -> str:
    s = (s or "").replace("\u00A0", " ").replace("\u200b", "")
    s = re.sub(r"[ \t]+", " ", s)
    return s.strip()

# Expresión regular que detecta signos de puntuación solo al INICIO o al FINAL de una cadena.
st_end_punt_re = re.compile(r"^[¡!¿?«»\"'()\[\]{}·•.,;:—–-]+|[¡!¿?«»\"'()\[\]{}·•.,;:—–-]+$")
def strip_punct(s: str) -> str:
    """
    Elimina signos de puntuación ubicados al inicio o al final de la cadena.
    No afecta la puntuación que está en el medio.

    Ejemplo:

    Entrada: "**¡ahijuelita!**"
    Salida:  "ahijuelita"
    """
    return st_end_punt_re.sub("", s or "").strip()


def clean_curs(s: str) -> str:
    """
    Quita el marcado de negrita y cursiva en Markdown, dejando solo el texto limpio.

    - Elimina `**` (negrita)
    - Elimina `__` (subrayado)
    - Reemplaza `*palabra*` por `palabra` (estaba en cursiva)

    Ejemplo:

    Entrada: "**¡ahijuelita!**  *bonita*"
    Salida:  "¡ahijuelita!  bonita"
    """
    s = (s or "").replace("**", "").replace("__", "")
    s = re.sub(r"\*(.*?)\*", r"\1", s)
    return s

# (por ejemplo: "palabra||", "texto |")
remove_trailing_pipes_pattern = re.compile(r"\s*\|+\s*$")

def remove_trailing_pipes(s: str) -> str:
    """
    Elimina las barras verticales '|' que aparecen al final de la cadena.
    También remueve los espacios que puedan estar antes de las barras.

    Ejemplo:

    Entrada: "ser vivo ||"
    Salida:  "ser vivo"

    Entrada: "abeja |"
    Salida:  "abeja"
    """
    return remove_trailing_pipes_pattern.sub("", s or "").strip()

# al final de un lema, que se usan en entradas como "bacano, a" o "cansado, da".
remove_gender_suffix_pattern = re.compile(r"\s*,\s*(?:a|da)\b\.?\s*$", re.IGNORECASE)

# Quitar sufijo de género ", a" o ", da"
def remove_gender_suffix(token: str) -> str:
    """
    Quita el sufijo literal ", a" o ", da" al final de un lema para quedarse con la forma base.
    Si no encuentra el sufijo, deja el texto sin cambios.

    Ejemplo:

    Entrada: "bacano, a"
    Salida:  "bacano"

    Entrada: "cansado, da"
    Salida:  "cansado"

    Entrada: "árbol"
    Salida:  "árbol"
    """
    before = token
    after = remove_gender_suffix_pattern.sub("", token)
    return (after.strip() or before.strip())

# Reemplazo de "~" por la palabra base (respetando sufijos y signos)
def replace_tilde_with_base(text: str, base: str) -> str:
    """
    Reemplaza la tilde "~" en una frase derivada por la palabra base del lema.
    Mantiene correctamente sufijos, espacios y signos de puntuación.

    Ejemplos:

    - Entrada: texto="~ de encostalados", palabra_base="carrera"
      Salida:  "carrera de encostalados"

    - Entrada: texto="echar ~", palabra_base="carreta"
      Salida:  "echar carreta"

    - Entrada: texto="hablar ~.", palabra_base="carreta"
      Salida:  "hablar carreta."

    - Entrada: texto="~osa", palabra_base="carranch"
      Salida:  "carranchosa"
    """
        
    if not text:
        return text
    text = re.sub(r"~([a-záéíóúüñ]+)", lambda m: base + m.group(1), text, flags=re.IGNORECASE)
    text = re.sub(r"~(\s+)([a-záéíóúüñ]+)", lambda m: base + m.group(1) + m.group(2), text, flags=re.IGNORECASE)
    text = re.sub(r"~([.,;:?!])", lambda m: base + m.group(1), text)
    text = text.replace("~", base)
    return text

# Encabezados con "||" (dentro del bloque en **...**)
def resolve_pipe_tilde_in_head(head_bold_content: str) -> str:

    """
    Resuelve el caso de encabezados que usan "||" (pipes) y tilde "~".
    Toma la parte DERECHA del "||" y reemplaza "~" por la palabra base que está a la izquierda.

    Ejemplos:

    - Entrada: lema_con_barras = "carrera || ~ de encostalados"
      Salida:  "carrera de encostalados"

    - Entrada: lema_con_barras = "carraca || echar ~"
      Salida:  "echar carraca"

    - Entrada: lema_con_barras = "carreta || echar o hablar ~"
      Salida:  "echar o hablar carreta"
    """
    parts = head_bold_content.split("||", 1)
    if len(parts) != 2:
        return None
    left, right = parts[0], parts[1]
    base = strip_punct(clean_curs(norm_spaces(left)))
    base = remove_gender_suffix(base)
    base = strip_punct(base)
    rhs = norm_spaces(right)
    rhs = replace_tilde_with_base(rhs, base)
    rhs = strip_punct(rhs)
    rhs = remove_trailing_pipes(rhs)
    rhs = norm_spaces(rhs)
    return rhs

# Extracción del lema en **negrita**

re_bold_head = re.compile(r"^-?\s*\*\*(.*?)\*\*")
def extract_bold_head_general(line: str) -> str:
    """
    Extrae el lema (palabra principal) que aparece encerrado entre **negritas** 
    al inicio de una línea en el diccionario.

    Ejemplos:

    - Entrada: linea = "- **carraca** f. coloq. Mandíbula del hombre o los animales."
      Salida:  "carraca"

    - Entrada: linea = "- **carreta || echar o hablar ~** fr. coloq. Hablar cosas triviales..."
      Salida:  "carreta || echar o hablar ~"

    - Entrada: linea = "- **carranchil (carranchín)** m. Enfermedad cutánea caracterizada por un fuerte escozor."
      Salida:  "carranchil (carranchín)"

    - Entrada: linea = "- **carrera || ~ de encostalados** Competencia deportiva en la cual los participantes..."
      Salida:  "carrera || ~ de encostalados"
    """
    m = re_bold_head.search(line)
    return m.group(1) if m else None

def remove_optional_plural_suffix(token: str) -> str:
    """
    Elimina sufijos de plural opcional "(s)" o "(es)" de un lema.

    Esto es común en el diccionario cuando un lema admite plural, 
    pero no se quiere que aparezca como parte de la palabra base.

    Ejemplos :
    
    - Entrada: "botarata(s)"  
      Salida:  "botarata"

    - Entrada: "carriel(es)"  
      Salida:  "carriel"

    - Entrada: "mataburro(s)"  
      Salida:  "mataburro"
    """
    return re.sub(r"\((?:s|es)\)$", "", token, flags=re.IGNORECASE)

def normalize_headword_content(head_content: str) -> str:
    """
    Normaliza el lema extraído de la cabecera en **negrita** de un artículo.

    Acciones realizadas:
    1. Normaliza caracteres Unicode a forma NFC.
    2. Si el lema contiene '||' (derivados con "~"), los resuelve
       reemplazando "~" por la palabra base.
    3. Elimina sufijos de plural opcional "(s)" o "(es)".
    4. Descarta variantes indicadas entre paréntesis después del lema.
    5. Elimina colas literales como ", a" o ", da" (marcas de género).
    6. Limpia signos de puntuación y barras verticales sobrantes.
    7. Normaliza espacios en blanco.

    Ejemplos :
    
    - Entrada: "carranchil (carranchín)"
      Salida:  "carranchil"

    - Entrada: "carrera || ~ de encostalados"
      Salida:  "carrera de encostalados"

    - Entrada: "botarata(s)"
      Salida:  "botarata"
    """
    head = nfc(head_content)
    if "||" in head:
        resolved = resolve_pipe_tilde_in_head(head)
        if resolved:
            return nfc(remove_optional_plural_suffix(resolved))
    token = norm_spaces(head)
    token = remove_optional_plural_suffix(token)
    token = token.split(" (", 1)[0]
    token = remove_gender_suffix(token)
    token = strip_punct(token)
    token = remove_trailing_pipes(token)
    token = norm_spaces(token)
    return nfc(token)

# Ocurrencias externas "|| **...~...**"
external_occurrence_pattern = re.compile(
    r"\|\|\s*\*{2}\s*(.*?)\s*\*{2}\s*([^|]*)(?=(?:\|\||$))",
    flags=re.DOTALL
)

# Eliminar todas las abreviaturas gramaticales y bloques de regiones
grammatical_tags  = [
    "m", "f", "pl", "sing", "sust",
    "adj", "adv", "intr", "tr", "prnl", "fr", "interj",
    "coloq", "pop", "rur", "vulg", "despect", "obsol"
]

# Patrón para reconocer combinaciones de género "m. y f."
gender_pattern  = r"(?:m(?:\s*\.)?\s*(?:y\s*f(?:\s*\.)?)?)"

# Patrón para identificar expresiones como "U. t. c. adj./s./prnl."
utc_pattern_PATTERN = r"U\s*\.\s*t\s*\.\s*c\s*\.\s*(?:s|adj|prnl)\s*\.?"


# Elimina cosas como: "m.", "f.", "adj.", "intr.", "U. t. c. adj.", etc.
drop_tag = re.compile(
    r"(?:\b{mf}\b)|(?:\b(?:{base})(?:\s*\.)?\b)|(?:\b{utc_pattern}\b)".format(
        mf=gender_pattern, base="|".join(grammatical_tags ), utc_pattern=utc_pattern_PATTERN
    ),
    flags=re.IGNORECASE
)

# Bloques de REGIONES: SOLO si están EN CURSIVA (entre * ... *)
region_names = [
    "Amaz","Ant","Atl","Bog","Bol","Boy","Cald","Chocó","Choco","Córd","Cord",
    "Costa Atl","Costa Pacíf","Costa Pacif","Cund","Guaj","Magd","Nar","NStder","Stder",
    "Quind","Risar","Tol","Valle","Huila",
    "Amazonas","Antioquia","Atlántico","Atlantico","Bogotá","Bogota","Bolívar","Bolivar",
    "Boyacá","Boyaca","Caldas","Córdoba","Cordoba",
    "Costa Atlántica","Costa Atlantica","Costa del Pacífico","Costa del Pacifico",
    "Cundinamarca","La Guajira","Magdalena","Nariño","Narino",
    "Norte de Santander","Santander","Quindío","Quindio","Risaralda","Tolima",
    "Valle del Cauca","Cauca","Meta","Arauca","Casanare",
    "Putumayo","Caquetá","Caqueta","Llanos"
]

# Unión alternada escapada de todas las regiones para incrustar en regex.
region_pattern_union = "|".join(sorted(map(re.escape, region_names), key=len, reverse=True))

# Ejemplos válidos: *Ant., Cald., Valle.*   *Costa Atl., Cund.*
# Están en cursiva, pueden tener punto o no, y estar separadas por comas o conjunciones.
# NO se detectan si NO están en cursiva (entre asteriscos).
italic_region_block = re.compile(
    r"\*\s*(?:{REG})(?:\.)?(?:\s*(?:,|y|o|u)\s*(?:{REG})(?:\.)?)*\s*\*".format(REG=region_pattern_union),
    flags=re.IGNORECASE
)

# limpieza de citas con esa estructura
citation_w_paren_re = re.compile(
    r'(?:-\s*)?\(\s*(?:Carrasquilla|Le[oó]n Rey)\s*,\s*[IVXLCDM]+\s*(?:,\s*(?:copla\s*)?\d+)?\s*"?\s*\)?',
    flags=re.IGNORECASE
)

# citas entre comillas
citation_w_quote_re = re.compile(
    r'(?:-\s*)?[\"“”]\s*Le[oó]n Rey\s*,\s*[IVXLCDM]+\s*[\"“”]',
    flags=re.IGNORECASE
)


def remove_citations(text: str) -> str:
    """
    Elimina citas de obras/autor incrustadas en el texto con los formatos detectados.

    Borra patrones como:
    - (Carrasquilla, I, 150")
    - (Carrasquilla, II, 291")
    - (León Rey, II, copla 3932")
    - "León Rey, I"  /  “León Rey, I”

    Además, corrige espacios dobles y comas sobrantes antes de signos.
    """
    if not text:
        return ""
    out = citation_w_paren_re.sub("", text)
    out = citation_w_quote_re.sub("", out)
    out = re.sub(r"\s*,\s*(?=[.,;:])", "", out)
    out = re.sub(r"\s{2,}", " ", out)
    return norm_spaces(out)

def clean_regional_blocks_and_grammar_tags(text: str) -> str:
    """
    Limpia del texto:
    1) Bloques de regiones SOLO cuando están en cursiva *...* 
       (p. ej., *Cund., Boy., Nar.*; *Ant., Cald., Valle.*).
    2) Abreviaturas/etiquetas gramaticales con límites de palabra
       (m., f., adj., intr., tr., prnl., fr., interj., coloq., etc., y "U. t. c. ...").
    3) Citas bibliográficas mediante `strip_bibliographic_citations`.
    4) Puntuación/espacios sobrantes tras las limpiezas.

    - No afecta texto normal: no elimina "nar" dentro de "nariz", p ej., 
      porque los bloques regionales se borran solo si están en *cursiva*.
    """
    if not text:
        return ""
    out = text

    # 1) Limpiar citas
    out = remove_citations(out)

    # 2) Quitar bloques de regiones estrictamente en cursiva
    while True:
        new_out = italic_region_block.sub("", out)
        if new_out == out:
            break
        out = new_out

    # 3) Quitar abreviaturas gramaticales
    out = drop_tag.sub("", out)

    # 4) Limpieza de puntuación y espacios
    out = re.sub(r"\s*,\s*(?=[.,;:])", "", out)
    out = re.sub(r"\s*[.,;:—–-]\s*(?=[.,;:—–-])", " ", out)
    out = re.sub(r"\s{2,}", " ", out)
    out = norm_spaces(strip_punct(out))
    return out

# Detecta numeraciones en negrita tipo **2.**, **3.**
bold_number_pattern  = re.compile(r"\*\*\s*(\d+)\.\s*\*\*")

# Detecta numeraciones simples tipo 2. , 3. (sin negrita)
plain_number_pattern = re.compile(r"(?<!\d)(\d{1,2})\.\s")


def count_enumerations(full_text_after_head: str) -> int:
    """
    Cuenta cuántas definiciones enumeradas (2., 3., etc.) aparecen
    en el texto de un artículo después del lema.

    Se usa para duplicar las filas en el DataFrame por cada acepción enumerada.

    Ejemplos:
   - count_numbered_definitions("**2.** Segunda acepción. **3.** Tercera acepción.") -> 2
    - count_numbered_definitions("Definición simple sin numeración.") -> 0
    """
    count = 0
    for g in bold_number_pattern.findall(full_text_after_head):
        try:
            if int(g) >= 2: count += 1
        except: pass
    for g in plain_number_pattern.findall(full_text_after_head):
        try:
            if int(g) >= 2: count += 1
        except: pass
    return count

# detección de 1., 2., 3.
enumeration_marker_pattern = re.compile(r"(?:\*\*\s*(\d+)\.\s*\*\*|(?<!\d)(\d{1,2})\.\s)")


def extract_enumerated_senses_from_first_line(first_line: str, head_end_pos: int):

    """
    Extrae y separa las definiciones enumeradas (1., 2., 3., …) de la primera línea del artículo.

    Separa el texto en segmentos para cada acepción numerada, y de cada segmento
    extrae el significado y el ejemplo usando `extract_meaning_example_from_tail`.

    Parámetros:

    first_line : str
        Primera línea completa del artículo (incluyendo el lema y las definiciones).
    head_end_pos : int
        Índice en el que termina el lema en la línea.

    Retorna:

    list of tuple(str, str)
        Lista de tuplas (significado, ejemplo) para cada acepción.

    Ejemplos:
    
     - first_line = "- **carramán** m. coloq. Vehículo viejo. **2.** Persona vieja, acabada."
     -> split_enumerated_definitions_from_line(first_line, 11)
    [('Vehículo viejo.', 'Persona vieja, acabada.')]

     - first_line = "- **carreta** f. Carrete para hilos. **2.** Carretilla para materiales. **3.** Charla trivial."
     -> split_enumerated_definitions_from_line(first_line, 10)
    [('Carrete para hilos.', ''), ('Carretilla para materiales.', ''), ('Charla trivial.', '')]
    """
        
    tail_raw = (first_line or "")[head_end_pos:]
    tail = tail_raw
    cuts = []
    for m in enumeration_marker_pattern.finditer(tail):
        num = m.group(1) or m.group(2)
        try:
            if int(num) >= 2:
                cuts.append((m.start(), m.end()))
        except:
            pass
    segments = []
    if not cuts:
        segments = [tail]
    else:
        start = 0
        for (s, e) in cuts:
            segments.append(tail[start:s])
            start = e
        segments.append(tail[start:])
    senses = []
    for seg in segments:
        sig, ej = extract_meaning_example_from_tail(seg)
        senses.append((sig, ej))
    return senses

# Contenido significativo antes del primer "||", HEAD significa que está entre **...**
def extract_base_from_head(head_bold_content: str) -> str:

    """
    Extrae la parte base del lema que aparece ANTES del primer '||'.

    Se utiliza para normalizar la palabra principal,
    removiendo puntuación, cursivas y sufijos de género.

    Ejemplos:
    
    - extract_base_from_head("carrera || ~ de encostalados")
     ->'carrera'

    - extract_base_from_head("carranchil (carranchín)")
     ->'carranchil'
    """
    left = head_bold_content.split("||", 1)[0] if "||" in head_bold_content else head_bold_content
    base = strip_punct(clean_curs(norm_spaces(left)))
    base = remove_gender_suffix(base)
    base = strip_punct(base)
    return norm_spaces(base)


def check_meaning_before_pipes(first_line: str, head_end_pos: int) -> bool:
    """
    Verifica si hay contenido significativo (significado) ANTES del primer '||'
    en el texto que sigue al lema.

    Esto permite detectar si el lema principal tiene una definición propia,
    antes de listar ocurrencias derivadas con '||'.

    Ejemplo:
    
    - check_meaning_before_pipes("- **carraca** f. Mandíbula. || echar ~.", 11)
     -> True  # Tiene definición antes de '||'

    - check_meaning_before_pipes("- **carrera || ~ de encostalados** Competencia...", 7)
     -> False # No hay definición antes de '||'
    """
    tail = first_line[head_end_pos:]
    before = tail.split("||", 1)[0]
    clean = clean_regional_blocks_and_grammar_tags(before)
    clean = strip_punct(norm_spaces(clean))
    return bool(clean)

# Variantes entre paréntesis dentro del **lema** 
parenthetical_variants_pattern  = re.compile(r"\(([^)]+)\)")

def extract_parenthetical_variants(head_bold_content: str) -> list:
    """
    Extrae las variantes indicadas entre paréntesis en el lema principal.

    Ignora los casos en los que el paréntesis solo indica plural opcional (s/es).

    Ejemplos:
    
    - extract_variants_from_parentheses("carranchil (carranchín)")
      -> ['carranchín']

    - extract_variants_from_parentheses("carriel(es)")
      ->  # Ignora plural opcional
    """

    left = head_bold_content.split("||", 1)[0] if "||" in head_bold_content else head_bold_content
    m = parenthetical_variants_pattern .search(left)
    if not m:
        return []
    content = m.group(1)
    if re.fullmatch(r"(?:s|es)", content.strip(), flags=re.IGNORECASE):
        return []
    parts = re.split(r"\s*(?:,|/|;|\bo\b|\bu\b|\by\b)\s*", content, flags=re.IGNORECASE)
    variants = []
    for raw in parts:
        v = norm_spaces(raw)
        if not v: continue
        if re.fullmatch(r"(?:s|es)", v, flags=re.IGNORECASE): continue
        v = remove_gender_suffix(v)
        v = strip_punct(v)
        v = remove_trailing_pipes(v)
        v = norm_spaces(v)
        if v: variants.append(nfc(v))
    return variants

# Extracción de significado y ejemplo ----

# Patrón para localizar el primer texto en cursiva (normalmente el ejemplo)
first_italic_pattern  = re.compile(r"\*(.*?)\*")

def extract_meaning_example_after_head(first_line: str, head_end_pos: int):
    """
    Extrae el significado y el ejemplo de la PRIMERA LÍNEA de un artículo,
    considerando el texto que sigue al lema.

    Busca la primera frase en cursiva como el ejemplo.
    El resto antes de la cursiva se considera el significado.

    Ejemplo:
    
    - extract_meaning_and_example_from_headline( "- **carraca** f. Mandíbula del hombre. *Había perdido la carraca.*", 11)
      -> ('Mandíbula del hombre.', 'Había perdido la carraca.')
    """
    tail_raw = (first_line or "")[head_end_pos:]
    tail = clean_regional_blocks_and_grammar_tags(tail_raw)
    m = first_italic_pattern .search(tail)
    if m:
        significado = norm_spaces(strip_punct(tail[:m.start()]))
        ejemplo     = norm_spaces(strip_punct(m.group(1)))
    else:
        significado = norm_spaces(strip_punct(tail))
        ejemplo     = ""
    significado = clean_regional_blocks_and_grammar_tags(significado)
    ejemplo     = clean_regional_blocks_and_grammar_tags(ejemplo)
    return significado, ejemplo

def extract_meaning_example_from_tail(tail_raw: str):
    """
    Extrae el significado y el ejemplo desde el resto de texto de un artículo
    (sin el lema).

    Busca el primer texto en cursiva como ejemplo. Lo anterior es el significado.

    Ejemplo:

    - extract_meaning_and_example_from_tail( "Mandíbula del hombre. *Había perdido la carraca.*")
      -> ('Mandíbula del hombre.', 'Había perdido la carraca.')
    """
    tail = clean_regional_blocks_and_grammar_tags(tail_raw or "")
    m = first_italic_pattern .search(tail)
    if m:
        significado = norm_spaces(strip_punct(tail[:m.start()]))
        ejemplo     = norm_spaces(strip_punct(m.group(1)))
    else:
        significado = norm_spaces(strip_punct(tail))
        ejemplo     = ""
    significado = clean_regional_blocks_and_grammar_tags(significado)
    ejemplo     = clean_regional_blocks_and_grammar_tags(ejemplo)
    return significado, ejemplo

# Segmentación en LINEAS
# línea que comienza (opcionalmente con "- ") y sigue un bloque **...**
article_start_regex = re.compile(r"^\s*-?\s*\*\*")
articles, current = [], []
with open(path_source, "r", encoding="utf-8") as f:
    for raw in f:
        line = nfc(raw.rstrip("\n"))
        if article_start_regex.match(line):
            if current:
                articles.append("\n".join(current))
                current = []
        current.append(line) 
    if current:
        articles.append("\n".join(current))

#  Reescritura de “Véase **X**”
see_also_pattern  = re.compile(r"\bVéase\b\s*\*{2}\s*([^*]+?)\s*\*{2}", flags=re.IGNORECASE)


def _first_line_tail_after_head(first_line: str) -> str:
    """
    Devuelve el texto que sigue inmediatamente después del bloque de cabecera en **negrita**
    dentro de la PRIMERA LÍNEA de un artículo.

    Se asume que la cabecera ya cumple el formato "- **lema** ...".
    Si no hay cabecera detectada, devuelve cadena vacía.

    Ejemplos:
    
    - get_tail_after_head_from_first_line("- **carraca** f. Mandíbula. *Ejemplo*")
      -> " f. Mandíbula. *Ejemplo*"

    - get_tail_after_head_from_first_line("- **carreta || echar ~** fr. coloq. Hablar cosas triviales.")
       -> " fr. coloq. Hablar cosas triviales."
    """
    m = re_bold_head.search(first_line or "")
    if not m:
        return ""
    return (first_line or "")[m.end():]


In [ ]:


# usar mapa: lema_normalizado -> cola de su primera línea (para poder “pegarla” al que dice “Véase **X**”)

lemma_to_tail = {}
for art in articles:
    fl = art.split("\n", 1)[0]
    head = extract_bold_head_general(fl)
    if not head:
        continue
    lema_norm = normalize_headword_content(head)
    if lema_norm and lema_norm not in lemma_to_tail:
        lemma_to_tail[lema_norm] = _first_line_tail_after_head(fl)

# Reescritura de artículos que contienen “Véase **X**”
rewritten_articles = []
for art in articles:
    parts = art.split("\n", 1)
    fl = parts[0]
    rest = parts[1] if len(parts) > 1 else ""
    mh = re_bold_head.search(fl)
    if not mh:
        rewritten_articles.append(art)
        continue

    head_end = mh.end()
    tail = fl[head_end:]

    msee = see_also_pattern .search(tail)
    if not msee:
        rewritten_articles.append(art)
        continue

    target_raw = msee.group(1)
    target_norm = normalize_headword_content(target_raw)
    target_tail = lemma_to_tail.get(target_norm, "")

    if not target_tail:
        rewritten_articles.append(art)
        continue

    new_first_line = fl[:head_end] + " " + target_tail.strip()
    new_art = new_first_line if not rest else (new_first_line + "\n" + rest)
    rewritten_articles.append(new_art)

articles = rewritten_articles



En el pipeline, los “Requisitos 1 y 2” son dos condiciones que determinan cómo extraer el significado y el ejemplo del artículo según su estructura. Req. 1 se aplica cuando el artículo es simple: no tiene || (pipes) ni numeraciones (2., 3., …), de modo que el significado y el ejemplo se toman directamente de la primera línea después del lema y se asignan al lema (y a sus variantes). Req. 2 se aplica cuando no hay numeraciones pero sí existen ||, ya sea dentro del lema o como ocurrencias externas: en este caso, el significado y el ejemplo de la primera línea se asignan a cada frase derivada generada con || y ~; además, si el lema tiene significado antes del primer ||, también se crea una fila para él. Cuando hay numeraciones, se ignoran Req. 1 y Req. 2 y se procesa cada acepción enumerada por separado. En resumen, Req. 1 maneja artículos de una sola acepción sin derivados, mientras que Req. 2 maneja artículos sin numeraciones pero con derivados; ambos evitan la lógica más compleja de segmentar por numeraciones.

In [ ]:

# PIPELINE PRINCIPAL: de artículos en texto -> filas (palabra, significado, ejemplo)


# Este bloque recorre cada artículo ya preprocesado (segmentado en artículos,
# reescritas las remisiones “Véase **X**”, y con utilidades de limpieza listas),
# y construye la lista de filas 'rows' que luego se convierte a DataFrame.
#
# IDEAS CLAVE DEL PROCESAMIENTO:
# - Un “artículo” es un bloque de líneas que comienza con una línea cuyo inicio
#   contiene un lema en **negrita** (p. ej., "- **carraca** …").
# - La primera línea del artículo contiene el lema y, frecuentemente, una definición
#   inicial (opcional) y/o ocurrencias derivadas separadas con "||".
# - Las “ocurrencias externas” son frases adicionales en **negrita** dentro del artículo,
#   introducidas con "|| **...~...**", donde "~" se reemplaza por la base del lema.
# - Las “acepciones enumeradas” (2., 3., …) duplican filas: una por cada acepción.
# - Si no hay enumeraciones (solo una acepción) y hay pipes/tilde, el significado/ejemplo
#   extraído de la primera línea se reparte a todas las frases derivadas (Req. 2).
# - Se eliminan etiquetas gramaticales (m., f., adj., intr., …), bloques de regiones
#   (SOLO si están *en cursiva*, p.ej. *Cund., Nar.*), y citas bibliográficas.
# - NO se eliminan trozos de región si no están en cursiva; así previene borrar “Nar”
#   dentro de “nariz”, por ejemplo.
# - Se limpian signos de puntuación al borde, pipes colgantes, sufijos de género (", a" / ", da"),
#   y se normalizan espacios y Unicode (NFC).

rows = []  # ← acumulador final de registros {"palabra": ..., "significado": ..., "ejemplo": ...}

for art in articles:
    # 1) Toma SOLO la primera línea; ahí está el lema en **negrita**
    first_line = art.split("\n", 1)[0]

    # 2) Extraer el bloque **...** tal cual aparece en la primera línea (puede incluir "|| ~")
    head_content = extract_bold_head_general(first_line)
    if not head_content:
        # Si no hay lema en negrita, no es un artículo válido
        continue

    # 3) Normalizar el lema base:
    #    - Se resuelven pipes/tilde si están dentro del lema (p.ej. "**carreta || echar ~**")
    #    - Quitar plural opcional "(s)/(es)", sufijos ", a"/", da", puntas de puntuación, pipes colgantes
    #    - NFC + espacios normalizados
    palabra_head = normalize_headword_content(head_content)
    if not palabra_head:
        # Lema vacío tras limpieza -> no hay palabra; saltar
        continue

    # 4) Localizar fin del bloque **lema** en la línea para poder cortar
    m_head = re_bold_head.search(first_line)
    head_end = m_head.end() if m_head else 0

    # 5) Extraer todo lo que sigue al lema en el artículo (todas las líneas) para análisis:
    #    -  para detectar ocurrencias externas con "|| **...~...**"
    #    -  para detectar numeraciones (2., 3., ...)
    after_head_all = art[art.find(first_line) + head_end:]

    # 6) Flags para estructura del artículo
    has_external_pipes = ("||" in after_head_all)                      # ¿hay ocurrencias externas?
    head_has_meaning   = check_meaning_before_pipes(first_line, head_end) if m_head else False  # ¿hay sig. propio del lema?

    # 7) ¿Cuántas acepciones enumeradas hay (2., 3., …)? -> n_defs = 1 + n_extra_defs
    n_extra_defs = count_enumerations(after_head_all)
    n_defs = 1 + n_extra_defs

    # 8) Variantes en el lema (p.ej., "**carranchil (carranchín)**") -> crear filas también para cada variante
    variants = extract_parenthetical_variants(head_content)

    # 9) Clasificación de casos (Req. 1 y Req. 2):
    #    - is_req1_simple: SOLO una acepción, SIN pipes externos, y SIN pipes en el lema
    #      -> extraer significado/ejemplo de la primera línea y asignar SOLO al lema base
    #    - is_req2_simple: SOLO una acepción y HAY pipes (en lema o externos)
    #      -> extraer significado/ejemplo de la primera línea y asignarlo a TODAS las frases derivadas
    head_has_pipes = ("||" in head_content)
    is_req1_simple = (not has_external_pipes) and (n_extra_defs == 0) and (not head_has_pipes)
    is_req2_simple = (n_extra_defs == 0) and (head_has_pipes or has_external_pipes)

    # 10) Si aplica, se extrae significado/ejemplo de la PRIMERA línea (tras el lema), ya con limpieza de etiquetas/regiones/citas.
    sig_req1 = ej_req1 = ""
    sig_req2 = ej_req2 = ""
    if (m_head and is_req1_simple) or (m_head and is_req2_simple):
        sig, ej = extract_meaning_example_after_head(first_line, head_end)
        if is_req1_simple:
            sig_req1, ej_req1 = sig, ej
        if is_req2_simple:
            sig_req2, ej_req2 = sig, ej

    # 11) Si hay enumeraciones (2., 3., …), se separan las acepciones de la primera línea,  y se extrae (significado, ejemplo) por segmento.
    enumerated_senses = []
    if n_extra_defs > 0:
        enumerated_senses = extract_enumerated_senses_from_first_line(first_line, head_end)
        # “Colchón” por si se detectó menos segmentos de los esperados
        if len(enumerated_senses) < n_defs:
            enumerated_senses += [("", "")] * (n_defs - len(enumerated_senses))

    # 12) CREACIÓN DE FILAS PARA LA PALABRA BASE (y variantes):
    #     - Si NO es el caso “solo pipes externos sin sig. propio”,
    #       crea filas para el lema base (y para cada variante), una por acepción.
    #       (En caso de enumeración, se usa enumerated_senses[i];
    #        si no, se usa los sig/ej tomados en Req.1/Req.2 o vacío.)
    #     - Caso especial excluido: cuando hay pipes externos y NO hay significado antes del primer "||":
    #       en ese caso, el lema base no “merece” fila por sí solo; solo se listan derivados.
    if not (has_external_pipes and not head_has_meaning):
        # a) Filas para el lema base
        for i in range(n_defs):
            if n_extra_defs > 0:
                sig_i, ej_i = enumerated_senses[i]
            else:
                sig_i = (sig_req1 if is_req1_simple else (sig_req2 if is_req2_simple else ""))
                ej_i  = (ej_req1  if is_req1_simple else (ej_req2  if is_req2_simple else ""))
            rows.append({
                "palabra": palabra_head,
                "significado": sig_i,
                "ejemplo": ej_i
            })
        # b) Filas para cada variante entre paréntesis (si existen)
        for var in variants:
            for i in range(n_defs):
                if n_extra_defs > 0:
                    sig_i, ej_i = enumerated_senses[i]
                else:
                    sig_i = (sig_req1 if is_req1_simple else (sig_req2 if is_req2_simple else ""))
                    ej_i  = (ej_req1  if is_req1_simple else (ej_req2  if is_req2_simple else ""))
                rows.append({
                    "palabra": var,
                    "significado": sig_i,
                    "ejemplo": ej_i
                })

    # 13) CREACIÓN DE FILAS PARA OCURRENCIAS EXTERNAS (derivados con "|| **...~...**"):
    #     - Por cada "**...~...**" tras "||", se reemplaza "~" por la base del lema.
    #     - Si NO hay enumeraciones, se puede heredar sig/ej de la 1ª línea (Req. 2).
    if has_external_pipes:
        # Base del lema para expandir "~"
        base_raw = extract_base_from_head(head_content)

        # Recorre todas las ocurrencias: "**frase_con_tilde** cola_local"
        for bold_phrase, local_tail in external_occurrence_pattern.findall(after_head_all):
            # Reemplazar tilde y limpiar marca de estilo/puntuación/pipes al final
            phrase = replace_tilde_with_base(bold_phrase, base_raw)
            phrase = clean_curs(phrase)
            phrase = strip_punct(remove_trailing_pipes(norm_spaces(phrase))).rstrip(".")
            if not phrase:
                # Evitar crear filas vacías si la frase quedó sin contenido
                continue

            # Si NO hay enumeraciones, extraer sig/ej de la cola local (después de esa frase)
            sig_loc = ej_loc = ""
            if n_extra_defs == 0:
                sig_loc, ej_loc = extract_meaning_example_from_tail(local_tail)

            rows.append({
                "palabra": phrase,
                "significado": sig_loc,
                "ejemplo": ej_loc
            })


# CONSTRUCCIÓN DEL df
df = pd.DataFrame(rows).reset_index(drop=True)

# (páginas iniciales/ruido de la transcripción antes del primer artículo real).
if len(df) >= 3:
    df = df.iloc[3:].reset_index(drop=True)

print(df.head(10))


           palabra                                        significado  \
0            abagó  Selección de los mejores frutos de la cosecha ...   
1          abalear                Disparar balas sobre alguien o algo   
2          abalear                            Herir o matar a balazos   
3           abaleo                                            Tiroteo   
4          abanico                               Ventilador eléctrico   
5          abanico  Utensilio rústico, hecho de fibras vegetales e...   
6       abaniquear                          Darse aire con un abanico   
7        abarrotes                                            Víveres   
8  ser (una) abeja                   Ser muy vivo, listo, aprovechado   
9         abombado                           Que es ligeramente tonto   

                                             ejemplo  
0  Si yo supiera lo cierto/Cuál jue el que te aco...  
1                                                     
2                              

In [ ]:
out_file_csv = "Lexicc_CaroCuervoFinal.csv"

# 1. Limpieza final de columnas
for col in ['palabra', 'significado', 'ejemplo']:
    # Convertir a string y limpiar espacios en extremos
    df[col] = df[col].astype(str).str.strip()
    

    df[col] = df[col].str.replace(r'^[\'"]|[\'"]$', '', regex=True)
    
    # Convertir todo a minúsculas
    df[col] = df[col].str.lower()

# 2. Ordena el DataFrame alfabéticamente por la columna 'palabra'
df = df.sort_values(by='palabra', ascending=True).reset_index(drop=True)

# 3. a CSV por comas
df.to_csv(out_file_csv, index=False, quoting=1)


